In [1]:
import win32com.client
import datetime
import pandas as pd
import re

### I. Automate 'New Term" email notifications

#### 1. Take a look at 'New Term" email to understand Regex structure to use

In [ ]:
def peek_outlook_emails_term_noti():
    # Create Outlook application object
    outlook_app = win32com.client.Dispatch('Outlook.Application')

    # Get Namespace
    namespace = outlook_app.GetNamespace("MAPI")

    # Get Inbox folder
    inbox = namespace.GetDefaultFolder(6)  # 6 corresponds to the Inbox folder

    # Specify subfolder name
    subfolder_l1_name = "1. STAFFS"
    subfolder_l2_name = "1.2 New Hire - Term"
    
    try:
        # Get the "New Hire" subfolder
        subfolder_l1 = inbox.Folders[subfolder_l1_name]
        subfolder_l2 = subfolder_l1.Folders[subfolder_l2_name]
    
    except Exception as e:
        print(f"Error accessing subfolder: {e}")
        return

    # Sender and title criteria
    sender_email = 'hr@thefirm.com'
    email_title_keyword = 'Leaving The Firm'
    
    # List to store email bodies
    email_bodies = []
    
    # Counter to limit the loop to 5 emails
    email_counter = 0

    # Iterate through items in the subfolder
    for item in subfolder_l2.Items:
        # Check if the item is a mail item
        if item.Class == 43:  # 43 corresponds to MailItem
            # Check sender and title criteria
            #if email_title_keyword.lower() in item.Subject.lower():
            if (sender_email.lower() in item.SenderEmailAddress.lower() 
               and email_title_keyword.lower() in item.Subject.lower()):
                                
                # Peek into content of selected email body
                email_bodies.append(item.Body)
                print(item.Body)
                
                 # Increment the counter
                email_counter += 1

                # Break out of the loop if the counter reaches 5
                if email_counter >= 5:
                    break

    # Return the list of email bodies
    return email_bodies

In [ ]:
# Run peek function to take a look at 5 example emails
peek_outlook_emails_term_noti()

#### 2. Codes to extract needed info from 'New Term' email notifications

In [ ]:
def extract_employee_information_term_noti(body):
    # Define regular expressions for extracting information
    structure = {
 
        'employee_name': r'\*\tEmployee Name:\s*(.+?)$',
        
        'employee_id': r'\*\tEmployee ID:\s*([\w]+)\s*$',
        
        'employment_type': r'\*\tEmployment Type:\s*(.+?)$',
        
        'last_day_worked': r'\*\tLast Day Worked:\s*([^(\n]+?)(?=\s*\(m/d/y\)|$)',
          
        'last_day_employment': r'\*\tLast Day of Employment:\s*([^(\n]+?)(?=\s*\(m/d/y\)|$)',
        
        'department': r'\*\tDepartment:\s*(.+?)$',
        
        #Work Location has special "Zero Width Space" (ZWSP) after *tab
        'work_location': r'Work Location:\s*(.+?)$',
        
        'manager': r'\*\tManager:\s*(.+?)$',
        
        'personal_email': r'\*\tPersonal Email:\s*(.+?)$',
        
        'company_email': r'\*\tCompany Email:\s*(.+?)$'
    }
       
    # Initialize dictionary to store extracted information
    employee_info = {}

    # Iterate over patterns and extract information for each line
    for key, pattern in structure.items():
        match = re.search(pattern, body, re.MULTILINE)
        if match:
            # Get the captured group from the match
            employee_info[key] = match.group(1).strip()

    return employee_info

def access_outlook_emails_term_noti():
    # Create Outlook application object
    outlook_app = win32com.client.Dispatch('Outlook.Application')

    # Get Namespace
    namespace = outlook_app.GetNamespace("MAPI")

    # Get Inbox folder
    inbox = namespace.GetDefaultFolder(6)  # 6 corresponds to the Inbox folder

    # Specify subfolder name
    subfolder_l1_name = "1. STAFFS"
    subfolder_l2_name = "1.2 New Hire - New Term"

    try:
        # Get the "New Hire - New Term" subfolder
        subfolder_l1 = inbox.Folders[subfolder_l1_name]
        subfolder_l2 = subfolder_l1.Folders[subfolder_l2_name]
    
    except Exception as e:
        print(f"Error accessing subfolder: {e}")
        return

    # Sender and title criteria
    sender_email = 'hr@thefirm.com'
    email_title_keyword = 'Leaving The Firm'
    
    # Create an empty dictionary to store email information
    email_data = {'Subject':[],                 'Received':[], 
                  'Employee Name':[],           'Employee ID':[],
                  'Employment Type':[],
                  'Last Day Worked': [],        'Last Day of Employment':[],  
                  'Department':[],              'Work Location': [],          
                  'Manager':[],                 'Personal Email':[],
                  'Company Email':[]}

    # Iterate through items in the subfolder
    for item in subfolder_l2.Items:
        # Check if the item is a mail item
        if item.Class == 43:  # 43 corresponds to MailItem
            # Check sender and title criteria
            if (sender_email.lower() in item.SenderEmailAddress.lower() 
                and item.Subject.lower().startswith(email_title_keyword.lower())):
                
                # Extract employee information from the email body
                employee_info = extract_employee_information_term_noti(item.Body)
                
               # Append email information to the Dictionary
                email_data['Subject'].append(item.Subject)
                email_data['Received'].append(item.ReceivedTime.strftime('%Y-%m-%d %H:%M:%S') if item.ReceivedTime else None)
                email_data['Employee Name'].append(employee_info.get('employee_name', None))
                email_data['Employee ID'].append(employee_info.get('employee_id', None))
                email_data['Employment Type'].append(employee_info.get('employment_type', None))
                email_data['Last Day Worked'].append(employee_info.get('last_day_worked', None))
                email_data['Last Day of Employment'].append(employee_info.get('last_day_employment', None))
                email_data['Department'].append(employee_info.get('department', None))
                email_data['Work Location'].append(employee_info.get('work_location', None))
                email_data['Manager'].append(employee_info.get('manager', None))
                email_data['Personal Email'].append(employee_info.get('personal_email', None))
                email_data['Company Email'].append(employee_info.get('company_email', None))
    
    # Create DataFrame from the collected email information previously stored as dictionary
    df = pd.DataFrame(email_data)
    return df

### RUN FUNCTIONS
df_new_term = access_outlook_emails_term_noti()

#Add column Department Code by extracting first 3 digits of "department name"
df_new_term["Department Code"] = df_new_term["Department"].apply(lambda x: "(None)" if "(None)" in x else x[:3])

# Sort DataFrame by "Receive Time"
df_new_term_sorted_by_receive_time = df_new_term.sort_values(by='Received', ascending=False)
display(df_new_term_sorted_by_receive_time)

### II. Automate 'New Hire" email notifications

#### 1. Take a look at 'New Hire" email to understand Regex structure to use

In [ ]:
def peek_outlook_emails_hire_noti():
    # Create Outlook application object
    outlook_app = win32com.client.Dispatch('Outlook.Application')

    # Get Namespace
    namespace = outlook_app.GetNamespace("MAPI")

    # Get Inbox folder
    inbox = namespace.GetDefaultFolder(6)  # 6 corresponds to the Inbox folder

    # Specify subfolder name
    subfolder_l1_name = "1. STAFFS"
    subfolder_l2_name = "1.2 New Hire - Term"


    try:
        # Get the "New Hire" subfolder
        subfolder_l1 = inbox.Folders[subfolder_l1_name]
        subfolder_l2 = subfolder_l1.Folders[subfolder_l2_name]
    
    except Exception as e:
        print(f"Error accessing subfolder: {e}")
        return

    # Sender and title criteria
    sender_email = 'HR@thefirm.com'
    email_title_keyword = 'New Hire'
    
    # List to store email bodies
    email_bodies = []
    
    # Counter to limit the loop to 5 emails
    email_counter = 0

    # Iterate through items in the subfolder
    for item in subfolder_l2.Items:
        # Check if the item is a mail item
        if item.Class == 43:  # 43 corresponds to MailItem
            # Check sender and title criteria
            #if email_title_keyword.lower() in item.Subject.lower():
            if (sender_email.lower() in item.SenderEmailAddress.lower() 
               and email_title_keyword.lower() in item.Subject.lower()):
                                
                # Peek into content of selected email body
                # Append the email body to the list
                email_bodies.append(item.Body)
                print(item.Body)
                
                 # Increment the counter
                email_counter += 1

                # Break out of the loop if the counter reaches 5
                if email_counter >= 5:
                    break

    # Return the list of email bodies
    return email_bodies

In [ ]:
# Run peek function to take a look at 5 example emails
peek_outlook_emails_hire_noti()

#### 2. Codes to extract needed info from 'New Hire' email notifications

In [ ]:
def access_outlook_emails_new_hire_noti():
    # Create Outlook application object
    outlook_app = win32com.client.Dispatch('Outlook.Application')

    # Get Namespace
    namespace = outlook_app.GetNamespace("MAPI")

    # Get Inbox folder
    inbox = namespace.GetDefaultFolder(6)  # 6 corresponds to the Inbox folder

    # Specify subfolder name
    subfolder_l1_name = "1. STAFFS"
    subfolder_l2_name = "1.2 New Hire - New Term"

    try:
        # Get the "New Hire - New Term" subfolder
        subfolder_l1 = inbox.Folders[subfolder_l1_name]
        subfolder_l2 = subfolder_l1.Folders[subfolder_l2_name]
    
    except Exception as e:
        print(f"Error accessing subfolder: {e}")
        return

    # Sender and title criteria
    sender_email = 'hr@thefirm.com'
    email_title_keyword = 'New Hire'
    
    # Create an empty dictionary to store email information
    email_data = {'Subject':[],                 'Received':[], 
                  'Employee Name':[],           'Start Date':[],      
                  'Employee Type': [],          'Work Location':[],  
                  'Region':[],                  'Function': [],          
                  'Department':[],              'Manager':[],
                  'Pay Currency':[]}

    # Iterate through items in the subfolder
    for item in subfolder_l2.Items:
        # Check if the item is a mail item
        if item.Class == 43:  # 43 corresponds to MailItem
            # Check sender and title criteria
            if sender_email.lower() in item.SenderEmailAddress.lower() and email_title_keyword.lower() in item.Subject.lower():
                
                # Extract employee information from the email body
                employee_info = extract_employee_information_new_hire_noti(item.Body)
                               
                # Append email information to the Dictionary
                email_data['Subject'].append(item.Subject)
                email_data['Received'].append(item.ReceivedTime.strftime('%Y-%m-%d %H:%M:%S') if item.ReceivedTime else None)
                email_data['Employee Name'].append(employee_info.get('employee_name', None))
                email_data['Start Date'].append(employee_info.get('start_date', None))                
                email_data['Employee Type'].append(employee_info.get('employee_type', None))               
                email_data['Work Location'].append(employee_info.get('work_location', None))                
                email_data['Region'].append(employee_info.get('region', None))
                email_data['Function'].append(employee_info.get('function', None))                
                email_data['Department'].append(employee_info.get('department', None))
                email_data['Manager'].append(employee_info.get('manager', None))
                email_data['Pay Currency'].append(employee_info.get('pay_currency', None))
    
    # Create DataFrame from the collected email information previously stored as dictionary
    df = pd.DataFrame(email_data)
    return df

### RUN FUNCTIONS
df_new_hire = access_outlook_emails_new_hire_noti()

#Add column Department Code by extracting first 3 digits of "department name"
df_new_hire["Department Code"] = df_new_hire["Department"].apply(lambda x: "(None)" if "(None)" in x else x[:3])

# Sort DataFrame by "Receive Time"
df_new_hire_sorted_by_receive_time = df_new_hire.sort_values(by='Received', ascending=False)
display(df_new_hire_sorted_by_receive_time)

### III. Write out result into 2 separate tabs in a new Excel spreadsheet

In [ ]:
# Save the DataFrame to an Excel file with today's date in the file name
current_datetime = datetime.datetime.now()

# Format the date as YYYY-MM-DD
today_date = current_datetime.strftime("%Y-%m-%d")

# Put today's date in file path
excel_file_path = f'G:/My Drive/Email Noti Extracts/Term_Hire_employee_info_{today_date}.xlsx'

# Create an Excel writer object
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    
    # Write New Term DataFrame to a tab in Excel
    df_new_term_sorted_by_receive_time.to_excel(writer, sheet_name='New Term', index=False)

    # Write New Hire DataFrame to a tab in Excel
    df_new_hire_sorted_by_receive_time.to_excel(writer, sheet_name='New Hire', index=False)

# Display the Excel file path
print(f"Excel file saved to: {excel_file_path}")